# Topic modelling

## 0.0 Imports

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dglover\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## 1.0 Run

In [4]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['use', 'show', 'however', 'approach', 'well', 'provide',' present', 'include', 'word', 'nlp', 'natural', 'language', 'p'])

In [5]:
# Import Dataset
df = pd.read_csv('..\\data\\50_summaries.csv', index_col=0)
#print(df.target_names.unique())
df.head()

,Summary
Filename,
1809.09190v1.pdf,In this paper we formulate audio play hot n c...
1903.10625v2.pdf,Finite State Transducers FST are an efficient...
1904.04307v1.pdf,The quantification of semantic similarity bet...
1809.02794v3.pdf,This paper focuses on the aim of semantic rol...
1805.01083v1.pdf,The mainconstructusedinextractionlanguagesand...


In [6]:
df.isna().sum()

Summary    0
dtype: int64

In [7]:
df.shape

(50, 1)

In [8]:
df['Summary'] = df['Summary'].map(lambda x: x.lower().replace('natural langauge processing', ''))

In [9]:
# Convert to list
data = df['Summary'].values.tolist()

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
#nlp = spacy.load('en', disable=['parser', 'ner'])
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

C:\Users\dglover\Anaconda3\lib\site-packages\spacy\language.py:1895: UserWarning: [W123] Argument disable with value ['parser', 'ner'] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  warnings.warn(


In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [26]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [27]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.026*"use" + 0.015*"method" + 0.013*"model" + 0.011*"network" + '
  '0.011*"evaluate" + 0.009*"task" + 0.009*"ground" + 0.009*"learn" + '
  '0.009*"traditional" + 0.009*"word"'),
 (1,
  '0.024*"language" + 0.015*"task" + 0.013*"use" + 0.013*"model" + '
  '0.010*"text" + 0.010*"natural" + 0.010*"processing" + 0.008*"organize" + '
  '0.006*"method" + 0.006*"word"'),
 (2,
  '0.029*"language" + 0.015*"use" + 0.015*"work" + 0.013*"task" + '
  '0.010*"framework" + 0.010*"lexical" + 0.010*"efficient" + 0.008*"semantic" '
  '+ 0.008*"pretraine" + 0.008*"context"'),
 (3,
  '0.019*"language" + 0.012*"system" + 0.012*"text" + 0.010*"interpretation" + '
  '0.010*"arabic" + 0.010*"task" + 0.010*"sentiment" + 0.010*"feature" + '
  '0.007*"use" + 0.007*"natural"'),
 (4,
  '0.035*"model" + 0.014*"language" + 0.012*"project" + 0.012*"key" + '
  '0.012*"base" + 0.012*"propose" + 0.009*"paper" + 0.009*"present" + '
  '0.009*"system" + 0.009*"task"')]


In [28]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.739581542479117

Coherence Score:  0.3423974426966424


In [29]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\dglover\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.130937  0.046639       1        1  25.458988
1     -0.022906 -0.054339       2        1  23.256846
0     -0.019819 -0.081584       3        1  18.635992
3      0.004411 -0.007200       4        1  17.032608
2     -0.092623  0.096484       5        1  15.615566, topic_info=         Term       Freq      Total Category  logprob  loglift
37      model  25.000000  25.000000  Default  30.0000  30.0000
99        key   5.000000   5.000000  Default  29.0000  29.0000
54        use  20.000000  20.000000  Default  28.0000  28.0000
67   evaluate   3.000000   3.000000  Default  27.0000  27.0000
105   project   6.000000   6.000000  Default  26.0000  26.0000
..        ...        ...        ...      ...      ...      ...
35      large   1.334030   5.992525   Topic5  -5.2457   0.3546
290  sentence   1.333733   6.535626   Topic5  -5.2459   0.2676
9       paper   1.334037   8.749515   Topic5  -5.2457  -0.0239
37      model   1.353206  25.590171   Topic5  -5.2314  -1.0828
191   propose   1.334646   9.525832   Topic5  -5.2452  -0.1084

[296 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
565       1  0.578964        achieve
565       3  0.289482        achieve
157       3  0.764413       actually
719       5  0.790401    adversarial
158       3  0.764413  agrawaletalet
...     ...       ...            ...
94        4  0.112209           work
94        5  0.448837           work
19        2  0.973783         worsen
379       1  0.570759          write
379       2  0.285379          write

[379 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 1, 4, 3])

In [30]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=texts)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

C:\Users\dglover\AppData\Local\Temp\ipykernel_7512\3153082663.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)


In [31]:
df_dominant_topic.to_csv('../data/Dominant_topics_of_absracts.csv')

In [32]:
df_dominant_topic.tail(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
40,40,4,0.9935,"model, language, project, key, base, propose, paper, present, system, task","[uniblock, scoring, filter, information, train, good, property, block, unsupervise, supervised, ..."
41,41,2,0.9908,"language, use, work, task, framework, lexical, efficient, semantic, pretraine, context","[troll, public, commurum, fix, emit, offensive, language, adversarial, attack, lead, developer, ..."
42,42,4,0.9917,"model, language, project, key, base, propose, paper, present, system, task","[base, contextual, emotion, classifier, contextual, emotion, classifier, base, emotion, propose,..."
43,43,0,0.9893,"use, method, model, network, evaluate, task, ground, learn, traditional, word","[program, synthesis, semantic, parsing, learn, code, idiom, author, require, highlevel, reasonin..."
44,44,1,0.9938,"language, task, use, model, text, natural, processing, organize, method, word","[unilateral, contract, term, service, play, substantial, role, modern, digital, life, propose, t..."
45,45,0,0.9946,"use, method, model, network, evaluate, task, ground, learn, traditional, word","[outofvocabulary, embed, imputation, ground, language, graph, convolutional, network, ground, in..."
46,46,3,0.9908,"language, system, text, interpretation, arabic, task, sentiment, feature, use, natural","[evaluation, basic, module, isolate, spelling, error, correction, polish, text, module, combine,..."
47,47,2,0.9920,"language, use, work, task, framework, lexical, efficient, semantic, pretraine, context","[pretraine, framework, language, understanding, framework, incrementally, build, pretraine, task..."
48,48,4,0.9909,"model, language, project, key, base, propose, paper, present, system, task","[new, model, achieve, considerable, deal, special, ever, application, gradually, introly, attent..."
49,49,4,0.9943,"model, language, project, key, base, propose, paper, present, system, task","[aim, build, neural, network, model, task, errorbase, errorcorrection, natural, language, applic..."


In [33]:
df_dominant_topic['Dominant_Topic'].value_counts()

4    13
1    10
0     9
3     9
2     9
Name: Dominant_Topic, dtype: int64

In [34]:
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

,Topic_Num,Topic_Perc_Contrib,Keywords,Text
0,0,0.9959,"use, method, model, network, evaluate, task, ground, learn, traditional, word","[quantification, semantic, similarity, word, use, evaluate, ability, system, perform, semantic, ..."
1,1,0.9961,"language, task, use, model, text, natural, processing, organize, method, word","[problem, tag, natural, language, processing, find, way, tag, text, meticulous, part, speech, te..."
2,2,0.9965,"language, use, work, task, framework, lexical, efficient, semantic, pretraine, context","[novel, teaching, parallel, distribute, computing, concept, use, memebase, programming, language..."
3,3,0.9959,"language, system, text, interpretation, arabic, task, sentiment, feature, use, natural","[grow, body, research, arabic, language, recent, year, example, orthography, dialect, make, sent..."
4,4,0.9965,"model, language, project, key, base, propose, paper, present, system, task","[automate, technique, model, check, violation, probability, reach, error, state, exceed, use, ve..."


In [35]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100

sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

,Topic_Num,Topic_Perc_Contrib,Keywords,Representative Text
0,0,0.9959,"use, method, model, network, evaluate, task, ground, learn, traditional, word","[quantification, semantic, similarity, word, use, evaluate, ability, system, perform, semantic, ..."
1,1,0.9961,"language, task, use, model, text, natural, processing, organize, method, word","[problem, tag, natural, language, processing, find, way, tag, text, meticulous, part, speech, te..."
2,2,0.9965,"language, use, work, task, framework, lexical, efficient, semantic, pretraine, context","[novel, teaching, parallel, distribute, computing, concept, use, memebase, programming, language..."
3,3,0.9959,"language, system, text, interpretation, arabic, task, sentiment, feature, use, natural","[grow, body, research, arabic, language, recent, year, example, orthography, dialect, make, sent..."
4,4,0.9965,"model, language, project, key, base, propose, paper, present, system, task","[automate, technique, model, check, violation, probability, reach, error, state, exceed, use, ve..."


In [36]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

,Dominant_Topic,Topic_Keywords,Num_Documents,Perc_Documents
0,1,"language, task, use, model, text, natural, processing, organize, method, word",9.0,0.18
1,1,"language, task, use, model, text, natural, processing, organize, method, word",10.0,0.20
2,0,"use, method, model, network, evaluate, task, ground, learn, traditional, word",9.0,0.18
3,4,"model, language, project, key, base, propose, paper, present, system, task",9.0,0.18
4,4,"model, language, project, key, base, propose, paper, present, system, task",13.0,0.26
5,3,"language, system, text, interpretation, arabic, task, sentiment, feature, use, natural",NaN,NaN
6,2,"language, use, work, task, framework, lexical, efficient, semantic, pretraine, context",NaN,NaN
7,0,"use, method, model, network, evaluate, task, ground, learn, traditional, word",NaN,NaN
8,4,"model, language, project, key, base, propose, paper, present, system, task",NaN,NaN
9,3,"language, system, text, interpretation, arabic, task, sentiment, feature, use, natural",NaN,NaN
